# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np

import json



from api_keys import geoapify_key




In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kathu,7.9178,98.3332,23.90,83,13,4.71,TH,1704754012
1,1,argo,37.6333,22.7333,9.32,80,89,1.09,GR,1704754012
2,2,egilsstadir,65.2653,-14.3948,5.95,65,0,4.12,IS,1704754013
3,3,macapa,0.0389,-51.0664,25.57,88,99,4.07,BR,1704754013
4,4,atafu village,-8.5421,-172.5159,29.01,71,10,3.59,TK,1704754014


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# Confirm the data types for the data columns are integers or floating-point decimal numbers.


city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [6]:

#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat",geo = True, tiles = "OSM", size = "Humidity", frame_width = 800, frame_height = 600, scale = 0.5, color = "City")


# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [7]:
# Get the max teperature.
max_temp =  city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [8]:
min_temp = city_data_df["Max Temp"] > 20
max_temp = city_data_df["Max Temp"] < 30
wind_speed = city_data_df["Wind Speed"] > 5

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
#Still using metric (C)

# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal

city_temp_range_df = city_data_df[min_temp & max_temp & wind_speed]

# Drop any rows with null values & Display sample data
city_temp_range_df.dropna()
city_temp_range_df.head()




,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,soure,-0.7167,-48.5233,26.85,85,100,5.89,BR,1704753778
13,13,adamstown,-25.0660,-130.1015,23.63,83,100,6.22,PN,1704754018
18,18,west island,-12.1568,96.8225,26.99,83,40,7.20,CC,1704754020
21,21,kailua-kona,19.6406,-155.9956,27.98,75,20,11.83,US,1704754021
22,22,avarua,-21.2078,-159.7750,28.03,78,75,7.72,CK,1704754022


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

new_types_df = pd.DataFrame(city_temp_range_df, columns = ["City", "Country", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] < 30) & (new_types_df["Max Temp"] > 20)
wind_speed = new_types_df["Wind Speed"] > 5
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]


# Display sample data
new_types_df.head()

hotel_df = new_types_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Max Temp,Wind Speed,Cloudiness,Hotel Name
7,soure,BR,26.85,5.89,100,
13,adamstown,PN,23.63,6.22,100,
18,west island,CC,26.99,7.20,40,
21,kailua-kona,US,27.98,11.83,20,
22,avarua,CK,28.03,7.72,75,
...,...,...,...,...,...,...
527,hobyo,SO,24.73,10.58,66,
550,saipan,MP,27.39,5.66,40,
558,uribia,CO,28.53,9.29,29,
559,eyl,SO,22.63,9.64,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
# Create a params dictionary
params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key}
print("Starting hotel search")
for index, row in hotel_df.iterrows():
    # Print a message to follow up the hotel search
   
    city = hotel_df.loc[index, "City"]
    # print(city)
    # Get latitude, longitude from the DataFrame
    lng = city_temp_range_df.loc[city_temp_range_df["City"] == city,'Lng'].item()
    # print(lng)
    lat = city_temp_range_df.loc[city_temp_range_df["City"] == city,'Lat'].item()
    # print(lat)
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
  
    params["bias"] = f"proximity:{lng},{lat}"
  
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
   
    # Make an API request using the params dictionary
    name_address_response = requests.get(base_url, params=params)
    # print(name_address_response)
    # Convert the API response to JSON format
    name_address = name_address_response.json()
    
    # print(name_address)
    
    if "features" in name_address and name_address["features"]:
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    else:
        # Print an error message or handle the error accordingly
        print(f"Error: {name_address_response.status_code}")# Grab the first hotel from the results and store the name in the hotel_df DataFrame
       
 
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

Starting hotel search
soure - nearest hotel: Hotel Marajó
adamstown - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
kailua-kona - nearest hotel: Kona Seaside Hotel
avarua - nearest hotel: Paradise Inn
mariental - nearest hotel: Mariental Hotel
lihue - nearest hotel: Kauai Palms
saltillo - nearest hotel: Hotel Colonial
kingston - nearest hotel: Aphrodite Hotel
arraial do cabo - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Hotel
palikir - national government center - nearest hotel: Mangrove Bay Hotel
nova sintra - nearest hotel: Residência Ka Dencho
pisco - nearest hotel: La Portada
vila velha - nearest hotel: Hotel Prainha
hadibu - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
manaure - nearest hotel: hotel palaima
san patricio - nearest hotel: No hotel found
enewetak - nearest hotel: No hotel found
tsiombe - nearest 

In [17]:
hotel_df


,City,Country,Max Temp,Wind Speed,Cloudiness,Hotel Name
7,soure,BR,26.85,5.89,100,Hotel Marajó
13,adamstown,PN,23.63,6.22,100,No hotel found
18,west island,CC,26.99,7.20,40,Cocos Village Bungalows
21,kailua-kona,US,27.98,11.83,20,Kona Seaside Hotel
22,avarua,CK,28.03,7.72,75,Paradise Inn
...,...,...,...,...,...,...
527,hobyo,SO,24.73,10.58,66,Iftin Hotel Hobyo
550,saipan,MP,27.39,5.66,40,Chalan Kanoa Beach Hotel
558,uribia,CO,28.53,9.29,29,No hotel found
559,eyl,SO,22.63,9.64,81,فندق ايل دابي


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size="Humidity",
    scale=0.5,
    color="City",
    hover_cols=["Hotel Name","Country"])
# Display the map plot
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)